# Lectura y uso de datos cualitativos

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import pymysql
import seaborn as sns
import requests
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup
import pandas as pd
import folium
from folium.plugins import HeatMap
from sklearn.linear_model import LinearRegression
from geopy import distance
from scipy import stats

# Posibilidades de investigacion con los datos
Temperatura ambiente y humedad: te permiten conocer las condiciones climáticas en las que se encuentra el ganado, lo que es importante para prevenir enfermedades, manejar el bienestar animal y ajustar la ventilación y calefacción en los establos.

Temperatura corporal del animal: la temperatura corporal del animal puede ser un indicador de su salud y bienestar. Puedes identificar patrones de temperatura que indiquen enfermedades, infecciones o estrés, lo que te permitirá actuar rápidamente para prevenir problemas de salud en el rebaño.

Ubicación geográfica: con la latitud y longitud de los animales, puedes realizar análisis de comportamiento y movimiento, como el análisis de pastoreo, que te permiten optimizar el uso de pastos y planificar la rotación del ganado.

Batería y fecha/hora: te permiten saber cuándo se realizó la última medición de los datos, lo que es importante para garantizar la precisión de los datos. Además, la información de batería te permite conocer el estado de los dispositivos y programar su reemplazo o carga.

Comportamiento del animal: la tecnología de seguimiento y monitoreo de animales también puede proporcionar información sobre el comportamiento del ganado, como la actividad y el tiempo de rumia. Esta información puede ayudarte a detectar signos tempranos de enfermedades, identificar patrones de comportamiento que indiquen estrés o cambios en el bienestar animal.

Edad del animal: conocer la edad de los animales puede ser importante para monitorear el crecimiento y desarrollo del ganado, y adaptar la alimentación y el manejo a las necesidades específicas de cada etapa de la vida.

Tiempo de permanencia en los corrales: registrar el tiempo que los animales pasan en los corrales es importante para monitorear la carga animal y prevenir el hacinamiento, lo que puede provocar estrés y enfermedades en el rebaño.

Identificación de animales: la identificación individual de los animales es importante para llevar un registro preciso de su historial de salud, reproducción y producción. Esto puede realizarse mediante el uso de dispositivos de identificación electrónica (EID) o sistemas de etiquetado con códigos de barras.

# Lectura de los datos

In [ ]:
# abro dataframe

df = pd.read_csv("./Dataset_TFn.csv")
df2 = pd.read_csv("./RE-ANALISIS-ANDRES-BASE-DATOS - copia.csv", sep=";")

'''url = "https://ganaderiadeprecision.net.ar"
data = {'pma_username': 'root',
        'pma_password': 'JAXEsCw5n9eXbrdv',
        'server': '1',
        'target': 'sql.php',
        'db': 'IoT',
        'table': 'Prueba',
        'set_session': 'f73085e0028564eac929b21e60d56550',
        'token': '3f3a5d7035496b67684b397e3d4e463d'}
session = requests.Session()
response = session.post(url, data=data)

html_content = response.text
soup = BeautifulSoup(html_content)
print(soup.prettify())'''

In [ ]:
df2

In [ ]:
df

# Acomodo de los datos para una mejor vision y practicidad

In [ ]:
df2.rename(columns={"Act.dia": "Act_dia"}, inplace=True)

In [ ]:
#Eliminar los datos no relevantes
#df = df[abs(df["t_Ambiente"] - df["t_Corporal"]) > 0.5]

In [ ]:
# Conversion de tipo int a fecha de pandas
df.isetitem(0,  pd.to_datetime(df["fecha"],format="%d/%m/%Y"))
df.isetitem(1,  pd.to_datetime(df["hora"]).dt.strftime('%H:%M'))

df2.isetitem(4,  pd.to_datetime(df2["DIA"],format="%d/%m/%Y"))
df2.isetitem(8,  pd.to_datetime(df2["HORA"]).dt.strftime('%H:%M'))
print(df)
print(df2)

In [ ]:
# Conversion de los datos de geolocalizacion a estandares internacionales
df.loc[:, "latitud"] = df["latitud"].apply(lambda x: x * -0.000001)
df.loc[:, "longitud"] = df["longitud"].apply(lambda x: x * -0.000001)
df2.loc[:, "x"] = df2["x"].apply(lambda x: x * -0.0001)
df2.loc[:, "y"] = df2["y"].apply(lambda x: x * -0.00001)
print(df)
print(df2)

In [ ]:
df2["EPOCA"] = df2["EPOCA"].astype("category")
df2["RAZA"] = df2["RAZA"].astype("category")
df2["Act_dia"] = df2["Act_dia"].astype("category")
df2.dtypes

In [ ]:
df2.EPOCA.unique()

In [ ]:
df2.RAZA.unique()

In [ ]:
df2.Act_dia.unique()

In [ ]:
df['season'] = df.loc[:,'fecha'].dt.month % 12 // 3 + 1
df['collar'] = 1
df

In [ ]:
df2['season'] = df2.loc[:,'DIA'].dt.month % 12 // 3 + 1
df2

In [ ]:
df.dtypes

In [ ]:
#Verificar si existen datos inexistente
df.isna().sum()

In [ ]:
df2 = pd.get_dummies(data=df2, columns=["EPOCA"], drop_first=True)
df2 = pd.get_dummies(data=df2, columns=["RAZA"], drop_first=True)
df2 = pd.get_dummies(data=df2, columns=["Act_dia"], drop_first=True)

La Epoca solo tiene 2 temporadas verano y invierno, por lo cual para futuro diseño de sistemas de prediccion se combierte verano en 1 y invierno en 0.
Lo mismo pasa con raza teniendo criolla argentina 1 y angus 0
Al contrario con la actividad que tiene 3 valores 'Descanso', 'Desplazamiento', 'Pastoreo', lo cual se crean 2 columnas desplazamiento y pastoreo lo cual tiene 1 si es lo realizado y 0 si no, al 2 tener 0 el animal se eencuetra en reposo

In [ ]:
df2

In [ ]:
df_temperatura = pd.DataFrame({"t_animal": df.loc[:,"t_Corporal"],
                           "t_ambiente": df.loc[:,"t_Ambiente"]})

In [ ]:
df_temperatura

In [ ]:
df_geo1 = pd.DataFrame({"Latitud": df.loc[:,"latitud"],
                       "Longitud":df.loc[:,"longitud"],
                       "Temporada": df.loc[:,"season"],
                       "Collar": df.loc[:,"collar"]})
df_geo2 = pd.DataFrame({"Latitud": df2.loc[:,"x"],
                       "Longitud":df2.loc[:,"y"],
                       "Temporada": df2.loc[:,"season"],
                       "Collar": df2.loc[:,"COLLAR"]})
df_geo = pd.concat([df_geo1,df_geo2], axis=0)
df_geo

In [ ]:
df_geo.isnull().any()

In [ ]:
stats.spearmanr(df_temperatura["t_ambiente"], df_temperatura["t_animal"])

In [ ]:
stats.spearmanr(df_geo["Latitud"], df_geo["Temporada"])

In [ ]:
stats.spearmanr(df_geo["Longitud"], df_geo["Temporada"])

In [ ]:
df.corr(min_periods=3)

In [ ]:
df2.corr(method='spearman',min_periods=3)

# Graficas

In [ ]:
#gráfico de dispersión que muestra la relación entre la temperatura ambiente y la temperatura corporal del animal

# Crear el gráfico de dispersión
plt.scatter(df['t_Ambiente'], df['t_Corporal'], color='red', marker='o')

# Ajustar una línea de tendencia
z = np.polyfit(df['t_Ambiente'], df['t_Corporal'], 1)
p = np.poly1d(z)
plt.plot(df['t_Ambiente'], p(df['t_Ambiente']), 'k--')

# Agregar etiquetas y título
plt.xlabel('Temperatura ambiente')
plt.ylabel('Temperatura del animal')
plt.title('Temperatura del animal en función de la temperatura ambiente')

# Mostrar el gráfico
plt.show()

In [ ]:
mapa = folium.Map(location=[df['latitud'].mean(), df['longitud'].mean()], zoom_start=13, )

folium.TileLayer(tiles='https://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', attr='Google', name='Google Satellite', max_zoom=20, subdomains=['mt0', 'mt1', 'mt2', 'mt3']).add_to(mapa)

# Añadir marcadores para cada ubicación
for i, row in df.iterrows():
    folium.Marker([row['latitud'], row['longitud']], popup=f"Humedad: {row['humedad']:.2f}\nFecha: {row['fecha']} {row['hora']}\n").add_to(mapa)

# Mostrar el mapa
mapa

In [ ]:
mapa = folium.Map(location=[df['latitud'].mean(), df['longitud'].mean()], zoom_start=13)

folium.TileLayer(tiles='https://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', attr='Google', name='Google Satellite', max_zoom=20, subdomains=['mt0', 'mt1', 'mt2', 'mt3']).add_to(mapa)

# Agregar marcadores al mapa
for index, row in df.iterrows():
    folium.Marker(
        location=[row['latitud'], row['longitud']],
        popup=f"Humedad: {row['humedad']}%<br>Fecha: {row['fecha']}<br>Hora: {row['hora']}",
        icon=folium.Icon(icon='info-sign', color='orange')
    ).add_to(mapa)
    
# Agregar capa de calor
heat_data = [[row['latitud'], row['longitud']] for index, row in df.iterrows()]
HeatMap(heat_data).add_to(mapa)

mapa

In [ ]:
df2['cantidad'] = df2.groupby(['x', 'y'])['x'].transform('count')

mapa = folium.Map(location=[df2['x'].mean(), df2['y'].mean()], zoom_start=13)

folium.TileLayer(tiles='https://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', attr='Google', name='Google Satellite', max_zoom=20, subdomains=['mt0', 'mt1', 'mt2', 'mt3']).add_to(mapa)

# Agregar capa de calor
heat_data = [[row['x'], row['y'], row['cantidad']] for index, row in df2.iterrows()]
HeatMap(heat_data).add_to(mapa)

# Mostrar el mapa
mapa

# Creacion de sistemas de prediccion